In [1]:
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/ferencipeter/Ironhack Data Analytics.json"

from google.cloud import bigquery

client = bigquery.Client()

In [5]:
query = '''
SELECT 
  tit.title_id,
  auth.au_id,
  tit.price * sale.qty * tit.royalty / 100 * titau.royaltyper / 100 AS sales_royalty
FROM 
  `ironhack-data-analytics.publications.authors` AS auth
JOIN
  `ironhack-data-analytics.publications.titleauthor` AS titau
  ON auth.au_id = titau.au_id
JOIN
  `ironhack-data-analytics.publications.titles` AS tit
  ON tit.title_id = titau.title_id
LEFT JOIN
  `ironhack-data-analytics.publications.publishers` AS pubs
  ON tit.pub_id = pubs.pub_id
LEFT JOIN
  `ironhack-data-analytics.publications.sales` AS sale
  ON tit.title_id = sale.title_id
LEFT JOIN
  `ironhack-data-analytics.publications.roysched` AS roy
  ON tit.title_id = roy.title_id
'''

query_job = client.query(query)

df=query_job.to_dataframe()
df.head(20)

,title_id,au_id,sales_royalty
0,TC3218,807-91-6654,83.800
1,TC3218,807-91-6654,83.800
2,TC3218,807-91-6654,83.800
3,TC3218,807-91-6654,83.800
4,TC3218,807-91-6654,83.800
5,TC3218,807-91-6654,83.800
6,TC3218,807-91-6654,83.800
7,TC3218,807-91-6654,83.800
8,MC3021,722-51-5454,13.455
9,MC3021,722-51-5454,13.455


In [7]:
query2 = '''
SELECT
  tit.title_id,
  auth.au_id,
  SUM(sales_royalty) AS royalties
FROM(
SELECT 
  tit.title_id AS Title,
  auth.au_id AS Author,
  tit.price * sale.qty * tit.royalty / 100 * titau.royaltyper / 100 AS sales_royalty
FROM 
  `ironhack-data-analytics.publications.authors` AS auth
JOIN
  `ironhack-data-analytics.publications.titleauthor` AS titau
  ON auth.au_id = titau.au_id
JOIN
  `ironhack-data-analytics.publications.titles` AS tit
  ON tit.title_id = titau.title_id
LEFT JOIN
  `ironhack-data-analytics.publications.publishers` AS pubs
  ON tit.pub_id = pubs.pub_id
LEFT JOIN
  `ironhack-data-analytics.publications.sales` AS sale
  ON tit.title_id = sale.title_id
LEFT JOIN
  `ironhack-data-analytics.publications.roysched` AS roy
  ON tit.title_id = roy.title_id
) royalty  
INNER JOIN `ironhack-data-analytics.publications.titles` AS tit
  ON tit.title_id = royalty.Title
INNER JOIN `ironhack-data-analytics.publications.authors` AS auth
  ON auth.au_id = royalty.Author
GROUP BY 
  1,2
'''

query_job = client.query(query2)

df=query_job.to_dataframe()
df.head(20)

,title_id,au_id,royalties
0,MC3021,722-51-5454,172.224
1,PS7777,486-29-1786,39.950
2,PC8888,846-92-7186,200.000
3,MC3021,899-46-2035,57.408
4,PS2091,899-46-2035,283.824
5,BU2075,213-46-8915,200.928
6,TC7777,267-41-2394,26.982
7,PS2091,998-72-3567,283.824
8,PC1035,238-95-7766,550.800
9,BU1111,267-41-2394,95.600


In [9]:
query3 = '''
SELECT
  auth.au_fname AS first_name,
  auth.au_lname AS last_name,
  auth.au_id AS author_id,
  SUM(royalty_total.royalties) + SUM(tit.advance * titau.royaltyper / 100) AS profit
FROM (
SELECT
  tit.title_id AS Title,
  auth.au_id AS Author,
  SUM(sales_royalty) AS royalties
FROM(
SELECT 
  tit.title_id AS Title,
  auth.au_id AS Author,
  tit.price * sale.qty * tit.royalty / 100 * titau.royaltyper / 100 AS sales_royalty
FROM 
  `ironhack-data-analytics.publications.authors` AS auth
JOIN
  `ironhack-data-analytics.publications.titleauthor` AS titau
  ON auth.au_id = titau.au_id
JOIN
  `ironhack-data-analytics.publications.titles` AS tit
  ON tit.title_id = titau.title_id
LEFT JOIN
  `ironhack-data-analytics.publications.publishers` AS pubs
  ON tit.pub_id = pubs.pub_id
LEFT JOIN
  `ironhack-data-analytics.publications.sales` AS sale
  ON tit.title_id = sale.title_id
LEFT JOIN
  `ironhack-data-analytics.publications.roysched` AS roy
  ON tit.title_id = roy.title_id
) royalty  
LEFT JOIN `ironhack-data-analytics.publications.titles` AS tit
  ON tit.title_id = royalty.Title
LEFT JOIN `ironhack-data-analytics.publications.authors` AS auth
  ON auth.au_id = royalty.Author
GROUP BY 
  1,2
) royalty_total
LEFT JOIN `ironhack-data-analytics.publications.authors` AS auth
  ON auth.au_id = royalty_total.Author
LEFT JOIN `ironhack-data-analytics.publications.titles` AS tit
  ON tit.title_id = royalty_total.Title
LEFT JOIN
  `ironhack-data-analytics.publications.titleauthor` AS titau
  ON auth.au_id = titau.au_id
GROUP BY
  1,2,3
ORDER BY
  profit DESC
LIMIT 3
'''

query_job = client.query(query3)

df=query_job.to_dataframe()
df.head(20)

,first_name,last_name,author_id,profit
0,Marjorie,Green,213-46-8915,21624.832
1,Anne,Ringer,899-46-2035,13638.714
2,Albert,Ringer,998-72-3567,13120.148


In [11]:
#that's probably not it

sql = '''
CREATE TABLE publications.author_ranking
SELECT
  auth.au_id AS author_id,
  SUM(royalty_total.royalties) + SUM(tit.advance * titau.royaltyper / 100) AS profit
FROM (
SELECT
  tit.title_id AS Title,
  auth.au_id AS Author,
  SUM(sales_royalty) AS royalties
FROM(
SELECT 
  tit.title_id AS Title,
  auth.au_id AS Author,
  tit.price * sale.qty * tit.royalty / 100 * titau.royaltyper / 100 AS sales_royalty
FROM 
  `ironhack-data-analytics.publications.authors` AS auth
JOIN
  `ironhack-data-analytics.publications.titleauthor` AS titau
  ON auth.au_id = titau.au_id
JOIN
  `ironhack-data-analytics.publications.titles` AS tit
  ON tit.title_id = titau.title_id
LEFT JOIN
  `ironhack-data-analytics.publications.publishers` AS pubs
  ON tit.pub_id = pubs.pub_id
LEFT JOIN
  `ironhack-data-analytics.publications.sales` AS sale
  ON tit.title_id = sale.title_id
LEFT JOIN
  `ironhack-data-analytics.publications.roysched` AS roy
  ON tit.title_id = roy.title_id
) royalty  
LEFT JOIN `ironhack-data-analytics.publications.titles` AS tit
  ON tit.title_id = royalty.Title
LEFT JOIN `ironhack-data-analytics.publications.authors` AS auth
  ON auth.au_id = royalty.Author
GROUP BY 
  1,2
) royalty_total
LEFT JOIN `ironhack-data-analytics.publications.authors` AS auth
  ON auth.au_id = royalty_total.Author
LEFT JOIN `ironhack-data-analytics.publications.titles` AS tit
  ON tit.title_id = royalty_total.Title
LEFT JOIN
  `ironhack-data-analytics.publications.titleauthor` AS titau
  ON auth.au_id = titau.au_id
GROUP BY
  1,2,3
ORDER BY
  profit DESC
LIMIT 3
'''

query_job = client.query(sql)